# MLFlow

End to end example integrating MLFlow with training and serving of models.

## Training

This first section covers how to train models using MLFlow.

### MLproject

The MLproject file defines:
- The environment where the training runs.
- The hyperparameters that can be tweaked. In our case, these are $\{\alpha, l_{1}\}$.
- The interface to train the model.

In [1]:
!ccat ./training/MLproject

name: mlflow-talk

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"


This allows us to have a single command to train the model. 

``` bash
$ mlflow run ./training -P alpha=... -P l1_ratio=...
```

In [2]:
!mlflow run ./training -P alpha=0.5

2019/11/04 13:54:07 INFO mlflow.projects: === Creating conda environment mlflow-1ecba04797edb7e7f7212d429debd9b664c31651 ===
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.5
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



tbb-2019.8           | 1.1 MB    | ##################################### | 100% 
setuptools-41.6.0    | 650 KB    | ##################################### | 100% 
tbb4py-2019.8        | 207 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Ran pip subprocess with arguments:
['/home/akash/miniconda3/envs/mlflow-1ecba04797edb7e7f7212d429debd9b664c31651/bin/python', '-m', 'pip', 'install', '-U', '-r', '/home/akash/open_source/av_dhs_demo/mlflow_workflow/training/condaenv.1nno8w4d.requirements.txt']
Pip subprocess output:
  Using cached https://files.pythonhosted.org/packages/

2019/11/04 13:54:40 INFO mlflow.projects: === Created directory /tmp/tmphh_ruw8v for downloading remote URIs passed to arguments of type 'path' ===
2019/11/04 13:54:40 INFO mlflow.projects: === Running command 'source /home/akash/miniconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-1ecba04797edb7e7f7212d429debd9b664c31651 1>&2 && python train.py 0.5 0.1' in run with ID 'b3a8b5eda5454b5ba42edd123ca1e51a' === 
Elasticnet model (alpha=0.500000, l1_ratio=0.100000):
  RMSE: 0.7947931019036529
  MAE: 0.6189130834228138
  R2: 0.18411668718221819
2019/11/04 13:54:43 INFO mlflow.projects: === Run (ID 'b3a8b5eda5454b5ba42edd123ca1e51a') succeeded ===


### MLtrack

The `train.py` script uses the `mlflow.log_param()` and `mlflow.log_metric()` commands to track each experiment. These are part of the `MLtrack` API, which tracks experiments parameters and results. These can be stored on a remote server, which can then be shared across the entire team. However, on our example we will store these locally on a `mlruns` folder.

In [3]:
!tree mlruns

mlruns
└── 0
    ├── b3a8b5eda5454b5ba42edd123ca1e51a
    │   ├── artifacts
    │   │   └── model
    │   │       ├── conda.yaml
    │   │       ├── MLmodel
    │   │       └── model.pkl
    │   ├── meta.yaml
    │   ├── metrics
    │   │   ├── mae
    │   │   ├── r2
    │   │   └── rmse
    │   ├── params
    │   │   ├── alpha
    │   │   └── l1_ratio
    │   └── tags
    │       ├── mlflow.project.backend
    │       ├── mlflow.project.entryPoint
    │       ├── mlflow.project.env
    │       ├── mlflow.source.git.commit
    │       ├── mlflow.source.name
    │       ├── mlflow.source.type
    │       └── mlflow.user
    └── meta.yaml

7 directories, 17 files


We can also run `mlflow ui` to show these visually. This will start the MLflow server in http://localhost:5000.

```bash
$ mlflow ui
```

In [4]:
!mlflow run ./training -P alpha=0.4

2019/10/30 10:46:46 INFO mlflow.projects: === Created directory /tmp/tmpli5tnl3g for downloading remote URIs passed to arguments of type 'path' ===
2019/10/30 10:46:46 INFO mlflow.projects: === Running command 'source /home/akash/miniconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-1ecba04797edb7e7f7212d429debd9b664c31651 1>&2 && python train.py 0.4 0.1' in run with ID '94a106dced9248989003ca6d200c2742' === 
Elasticnet model (alpha=0.400000, l1_ratio=0.100000):
  RMSE: 0.7909069124367867
  MAE: 0.6174288492244517
  R2: 0.19207580388574486
2019/10/30 10:46:48 INFO mlflow.projects: === Run (ID '94a106dced9248989003ca6d200c2742') succeeded ===


In [5]:
# !mlflow ui

[2019-10-29 22:52:05 +0530] [10249] [INFO] Starting gunicorn 19.9.0
[2019-10-29 22:52:05 +0530] [10249] [INFO] Listening at: http://127.0.0.1:5000 (10249)
[2019-10-29 22:52:05 +0530] [10249] [INFO] Using worker: sync
[2019-10-29 22:52:05 +0530] [10260] [INFO] Booting worker with pid: 10260
^C
[2019-10-29 22:53:00 +0530] [10249] [INFO] Handling signal: int
[2019-10-29 22:53:00 +0530] [10260] [INFO] Worker exiting (pid: 10260)


![MLFlow UI](./images/mlflow-ui.png)

### MLmodel

The `MLmodel` file allows us to version and share models easily. Below we can see an example.

In [7]:
!ccat ./mlruns/0/539b7e5225844c88a764baaa10112f00/artifacts/model/MLmodel

artifact_path: model
flavors:
  python_function:
    data: model.pkl
    env: conda.yaml
    loader_module: mlflow.sklearn
    python_version: 3.6.9
  sklearn:
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 0.19.1
run_id: 539b7e5225844c88a764baaa10112f00
utc_time_created: '2019-10-29 17:20:24.407684'


As we can see above the `MLmodel` keeps track, between others, of

- The experiment id, `539b7e5225844c88a764baaa10112f00`
- Date 
- Version of `sklearn` 
- How the model was stored

As we shall see shortly, the MLFlow model server will use this file to serve this model.

## Serving

### Set up

In [5]:
# !mlflow models serve -m ./mlruns/0/733d504633c745ddaf5f6ae4607d1022/artifacts/model -p 1234

2019/10/30 17:42:23 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2019/10/30 17:42:24 INFO mlflow.projects: === Creating conda environment mlflow-a0195768c92ab9f83c11ac37cf0491c5140e197d ===
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.5
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Ran pip subprocess with arguments:
['/home/akash/miniconda3/envs/mlflow-a0195768c92ab9f83c11ac37cf0491c5140e197d/bin/python', '-m', 'pip', 'install', '-U', '-r', '/home/akash/open_source/av_dhs_demo/mlruns/0/733d504633c745ddaf5f6ae4607d1022/artifacts/model/condaenv.z9mc_vut.requirements.txt']
Pip subprocess output:
Processing /home/akash/.cache/pip/wheels/f1/e0/1c/663c9b2bb00f32c235c98a43c39d81c2a25544a889a32649ea/mlflow-1.3.0-cp36-none-any.whl
  Using cached https://files.pythonhosted.or

2019/10/30 17:42:47 INFO mlflow.pyfunc.backend: === Running command 'source /home/akash/miniconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-a0195768c92ab9f83c11ac37cf0491c5140e197d 1>&2 && gunicorn --timeout=60 -b 127.0.0.1:1234 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2019-10-30 17:42:47 +0530] [19320] [INFO] Starting gunicorn 19.9.0
[2019-10-30 17:42:47 +0530] [19320] [INFO] Listening at: http://127.0.0.1:1234 (19320)
[2019-10-30 17:42:47 +0530] [19320] [INFO] Using worker: sync
[2019-10-30 17:42:47 +0530] [19329] [INFO] Booting worker with pid: 19329
/home/akash/miniconda3/envs/mlflow-a0195768c92ab9f83c11ac37cf0491c5140e197d/lib/python3.6/site-packages/mlflow/pyfunc/__init__.py:281: DeprecationWarning: .. Warning:: ``mlflow.pyfunc.load_pyfunc`` is deprecated since 1.0. This method will be removed in a near future release. Use ``mlflow.pyfunc.load_model`` instead.
  return load_pyfunc(model_uri, suppress_warnings)
^C
[2019-10-30 17:44:06 +053

#### Test models

We will now run a sample query to test that the deployment is working.

In [8]:
!curl -X POST -H "Content-Type:application/json; format=pandas-split" \
  --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://127.0.0.1:1234/invocations

[4.203048595214434]